## Part 1: Existing Machine Learning Services

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part1/score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Obtain labelled reviews

In order to test any of the sentiment analysis APIs, we need a labelled dataset of reviews and their sentiment polarity. We'll use NLTK to download the movie_reviews corpus.

In [1]:
from nltk import download

download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


True

### Load the data

The files in movie_reviews have already been divided into two sets: positive ('pos') and negative ('neg'), so we can load the raw text of the reviews into two lists, one for each polarity.

In [2]:
from nltk.corpus import movie_reviews

# extract words from reviews, pair with label

reviews_pos = []
for fileid in movie_reviews.fileids('pos'):
    review = movie_reviews.raw(fileid)
    reviews_pos.append(review)

reviews_neg = []
for fileid in movie_reviews.fileids('neg'):
    review = movie_reviews.raw(fileid)
    reviews_neg.append(review)

### Connect to the scoring API

Fill in this function with code that connects to the Amazon Comprehend API, and uses it to score a single review:

* [Documentation - Amazon Comprehend: Detect Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment.html)

Your function must return either 'pos' or 'neg', so you'll need to make some decisions about how to map the results of the API call to one of these values. Amazon Comprehend can return "NEUTRAL" or "MIXED" for the Sentiment - if this happens, you will need to inspect the numeric values under the SentimentScore to see whether it leans toward positive or negative.


In [16]:
from enum import Enum

import boto3

class Sentiment(Enum):
    """
    Enum representing the possible sentiment values returned by AWS Comprehend.
    """
    Positive = 'POSITIVE'
    Negative = 'NEGATIVE'
    Neutral = 'NEUTRAL'
    Mixed = 'MIXED'

def score_review(review, client):
    """
    Analyzes the sentiment of a review using AWS Comprehend.

    Parameters:
    review (str): The review text to analyze. If the review is longer than 5000 characters, it will be truncated.
    client (boto3.client): The AWS Comprehend client.

    Returns:
    str: The determined sentiment of the review. Possible values are 'pos' and 'neg'.

    Raises:
    botocore.exceptions.BotoCoreError: If there's an issue with the AWS Comprehend request.
    """
    LANGUAGE_CODE = 'en'
    
    # Limit review to 5000 characters.
    review = review[:5000]
    
    determined_sentiment = None
    
    response = client.detect_sentiment(
        Text=review, 
        LanguageCode=LANGUAGE_CODE
    )
    
    response_sentiment = response['Sentiment']
    
    if response_sentiment == Sentiment.Positive.value:
        determined_sentiment = 'pos'
    elif response_sentiment == Sentiment.Negative.value:
        determined_sentiment = 'neg'
    elif response_sentiment in [Sentiment.Neutral.value, Sentiment.Mixed.value]:
        # Check SentimentScore to determine positive or negative leaning
        positive_score = response['SentimentScore']['Positive']
        negative_score = response['SentimentScore']['Negative']

        if positive_score > negative_score:
            determined_sentiment = 'pos'
        elif negative_score > positive_score:
            determined_sentiment = 'neg'
    
    return determined_sentiment
    

### Score each review

Now, we can use the function you defined to score each of the reviews

In [20]:
import os
import boto3

# Load AWS credentials from environment variables.
aws_access_key_id = os.environ['AWS_ACCESS_KEY_ID']
aws_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY']
aws_region_name = os.environ['AWS_DEFAULT_REGION']

# Initialize client.
client = boto3.client('comprehend',
                    region_name=aws_region_name,
                    aws_access_key_id=aws_access_key_id,
                    aws_secret_access_key=aws_secret_access_key)

# Create 2 smaller subsets for testing
# subset_pos = reviews_pos[:10]
# subset_neg = reviews_neg[:10]

results_pos = []
# When comfortable with results switch `subset_pos` to reviews_pos`
for review in reviews_pos:
    result = score_review(review, client)
    print(result)
    results_pos.append(result)

results_neg = []
# When comfortable with results switch `subset_neg` to reviews_neg`
for review in reviews_neg:
    result = score_review(review, client)
    print(result)
    results_neg.append(result)

pos
neg
neg
pos
neg
pos
neg
pos
neg
pos
pos
neg
pos
pos
neg
pos
pos
pos
pos
pos
pos
pos
pos
pos
pos
neg
pos
pos
pos
pos
pos
neg
pos
pos
pos
pos
pos
neg
pos
pos
pos
pos
pos
neg
pos
neg
neg
pos
pos
neg
neg
pos
neg
neg
pos
pos
pos
neg
pos
pos
neg
pos
neg
neg
neg
pos
pos
neg
neg
pos
pos
neg
pos
pos
pos
neg
neg
pos
pos
neg
neg
pos
pos
neg
pos
pos
pos
pos
pos
pos
neg
pos
neg
neg
neg
neg
neg
pos
pos
neg
neg
pos
pos
neg
neg
neg
pos
neg
pos
pos
neg
neg
pos
neg
pos
neg
pos
neg
pos
neg
pos
neg
pos
neg
pos
pos
pos
pos
pos
pos
pos
neg
pos
pos
pos
pos
pos
pos
pos
neg
neg
neg
pos
pos
pos
pos
pos
pos
pos
neg
pos
pos
pos
neg
neg
pos
pos
pos
pos
pos
pos
pos
neg
pos
pos
pos
pos
pos
pos
pos
neg
neg
neg
neg
neg
pos
pos
neg
pos
neg
neg
pos
pos
pos
pos
pos
pos
pos
neg
neg
pos
pos
neg
pos
pos
pos
pos
neg
neg
pos
pos
pos
pos
pos
pos
neg
pos
pos
neg
pos
pos
neg
neg
pos
pos
neg
pos
pos
neg
neg
neg
pos
pos
neg
pos
neg
neg
pos
pos
pos
neg
neg
neg
pos
pos
pos
neg
pos
pos
pos
neg
neg
neg
pos
pos
pos
pos
pos
neg
pos


### Calculate accuracy

For each of our known positive reviews, we can count the number which our function scored as 'pos', and use this to calculate the % accuracy. We repeat
 this for negative reviews, and also for overall accuracy.

In [21]:
correct_pos = results_pos.count('pos')
accuracy_pos = float(correct_pos) / len(results_pos)
correct_neg = results_neg.count('neg')
accuracy_neg = float(correct_neg) / len(results_neg)
correct_all = correct_pos + correct_neg
accuracy_all = float(correct_all) / (len(results_pos)+len(results_neg))

print('Positive reviews: {}% correct'.format(accuracy_pos*100))
print('Negative reviews: {}% correct'.format(accuracy_neg*100))
print('Overall accuracy: {}% correct'.format(accuracy_all*100))

Positive reviews: 64.7% correct
Negative reviews: 68.0% correct
Overall accuracy: 66.35% correct
